In [3]:
import pandas as pd
import json

In [4]:
RELEVANT_CONCEPTS = ["ESA", "GP", "PV", "BDG", "WLSA", "SPC", "PRL", "PLF", "APC"]
csv_file_path = '../ham_concept_dataset/Datasets/metadata/metadata_ground_truth.csv'
df_ground_truth = pd.read_csv(csv_file_path)

json_file_path = '../cbm/skin_lesion_concept_abbreviations-4b.json'
with open(json_file_path, 'r') as f:
    json_data_eval = json.load(f)

total_comparisons = 0
matches = 0
mismatches_details = []

In [5]:
for index, gt_row in df_ground_truth.iterrows():
    image_id = gt_row['image_id']

    # Get concept values from JSON for this image
    eval_concepts_for_image = {}
    if image_id in json_data_eval:
        json_image_entry = json_data_eval[image_id]
        for concept_abbr in RELEVANT_CONCEPTS:
            eval_concepts_for_image[concept_abbr] = 1 if concept_abbr in json_image_entry else 0
    else:
        # If image_id is not in JSON, all its relevant concepts are considered 0 (absent)
        for concept_abbr in RELEVANT_CONCEPTS:
            eval_concepts_for_image[concept_abbr] = 0

    # Compare for each relevant concept
    for concept_abbr in RELEVANT_CONCEPTS:
        if concept_abbr not in df_ground_truth.columns:
            # This case should ideally not happen if the CSV is well-formed
            # and contains all relevant concept columns.
            # print(f"Warning: Concept {concept_abbr} not found as a column in CSV for image {image_id}. Skipping.")
            continue

        total_comparisons += 1
        gt_value = int(gt_row[concept_abbr])
        eval_value = eval_concepts_for_image[concept_abbr]

        if gt_value == eval_value:
            matches += 1
        else:
            mismatches_details.append({
                "image_id": image_id,
                "concept": concept_abbr,
                "ground_truth_csv": gt_value,
                "evaluation_json": eval_value
            })

# Calculate accuracy
accuracy = (matches / total_comparisons) * 100 if total_comparisons > 0 else 0


In [7]:
print(f"Comparison for concepts: {', '.join(RELEVANT_CONCEPTS)}\n")
print(f"Total image entries in CSV: {len(df_ground_truth)}")
print(f"Total individual concept comparisons made: {total_comparisons}")
print(f"Number of matches: {matches}")
print(f"Number of mismatches: {len(mismatches_details)}")
print(f"Accuracy: {accuracy:.2f}%")

Comparison for concepts: ESA, GP, PV, BDG, WLSA, SPC, PRL, PLF, APC

Total image entries in CSV: 6498
Total individual concept comparisons made: 58482
Number of matches: 49684
Number of mismatches: 8798
Accuracy: 84.96%
